# 전처리

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
from tqdm import trange

In [ ]:
BASE_DIR = "."

In [ ]:
def crop(img_num):
    file = f"{BASE_DIR}/dataset/Item-Image/img{img_num}.jpg"
    img1 = cv2.imread(file)
    
    # Image Shape
    h, w = img1.shape[:2]
    h1, h2 = int(h * 0.2), int(h * 0.7)
    w1, w2 = int(w * 0.05), int(w * 0.95)
    img = img1[h1: h2, w1: w2]
    
    # Resolution
    img = cv2.resize(img, None, fx=0.75, fy=0.75, interpolation=cv2.INTER_AREA)
    
    # Gray Scale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Adaptive Threshold
    img2 = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)
    
    #Blur
    blur = cv2.medianBlur(img2, 5, dst=None)
    
    # Add Kernel
    kernel = np.ones((3,3),np.uint8)
    
    # Morphology
    closing = cv2.morphologyEx(blur, cv2.MORPH_CLOSE, kernel)
    gradient = cv2.morphologyEx(closing, cv2.MORPH_GRADIENT, kernel)
    
    # 경계값 도출
    contours = cv2.findContours(gradient, cv2.RETR_TREE, cv2.CHAIN_APPROX_TC89_L1)[0]

    x1 = [] #x-min
    y1 = [] #y-min
    x2 = [] #x-max
    y2 = [] #y-max
    for i in range(1, len(contours)):
        ret = cv2.boundingRect(contours[i])
        x1.append(ret[0])
        y1.append(ret[1])
        x2.append(ret[0] + ret[2])
        y2.append(ret[1] + ret[3])

    x1_min = min(x1)
    y1_min = min(y1)
    x2_max = max(x2)
    y2_max = max(y2)
    cv2.rectangle(gradient, (x1_min, y1_min), (x2_max, y2_max), (0, 255, 0), 3)

    origial_img = gray[y1_min:y2_max, x1_min:x2_max]
    crop_img = img2[y1_min:y2_max, x1_min:x2_max]
    
    crop_img = cv2.resize(crop_img, (32, 32))
    
    #img_merge = np.hstack((origial_img, crop_img))
    
    #return img_merge
    
    return crop_img

#img resize test
img = crop(14)
plt.imshow(img, cmap='gray')

In [ ]:
def labeling(img_num):
    with open(f"{BASE_DIR}/dataset/Item-Parse_f/img{img_num}.json", "r", encoding="utf8") as f:
        contents = f.read()
        json_data = json.loads(contents)
        #label = json_data["category_name"]
        label = json_data["category_id"]
        
        return label

In [ ]:
outlier_list = []

In [ ]:
f = open(f"{BASE_DIR}/outlier.txt", "r")
lines = f.readlines()
for line in lines:
    line = line.strip()
    outlier_list.append(int(line))
f.close()
print(len(outlier_list))

In [ ]:
def outlier(img_num):
    for i in outlier_list:
        if i == img_num:
            return -1
    return 0

In [ ]:
def test(count):
    for img_num in range(count):
        crop_img = crop(img_num)
        #print(crop_img)
        label = labeling(img_num)

        plt.imshow(crop_img, cmap='gray')
        plt.title(f'img{img_num} {label}')
        plt.show()

In [ ]:
def save(img_num): # 이미지 번호를 인자로 받고 크롭, 라벨링 후 파일 저장
    img = crop(img_num)
    label = labeling(img_num)
    if outlier(img_num) == -1:
        save_src = f"{BASE_DIR}/labeling/outlier/{label}/img{img_num}.jpg"
        print(f"img{img_num} saved, outlier")
    else:
        save_src = f"{BASE_DIR}/labeling/{label}/img{img_num}.jpg"
        print(f"img{img_num} saved")
    
    cv2.imwrite(save_src, img)

In [ ]:
def save_run(count): #이미지 수를 인자로 받음
  for img_num in range(count):
    save(img_num)

In [ ]:
# 배열 생성
xDatas = []
yDatas = []

In [ ]:
#배열에 이미지 추가
def insertDatas():
    for imgNum in trange(30000):
        if outlier(imgNum) == -1:
            print(f"img{imgNum} continue")
            continue
        img = crop(imgNum)
        label = labeling(imgNum)
        xDatas.append(img)
        yDatas.append(label)

In [ ]:
insertDatas()

#배열을 np 배열로 전환
xDatas = np.array(xDatas)
yDatas = np.array(yDatas)

#np 배열을 파일로 저장
np.save(f'{BASE_DIR}/xDatas32_outlier', xDatas)
np.save(f'{BASE_DIR}/yDatas_outlier', yDatas)

In [ ]:
#np 배열 불러오기
xDatas = np.load(f'{BASE_DIR}/xDatas32_outlier.npy', allow_pickle=True)
yDatas = np.load(f'{BASE_DIR}/yDatas_outlier.npy')

#np 배열 개수 로딩
print(len(xDatas))
print(len(yDatas))